In [ ]:
# Inputs: τ and R0
# Result: the p value which produces outbreaks with the input R0 and τ value
# The resulting data is plotted and overlayed on the log-scale phase diagram as iso-R0 lines.

# This code is written to be run on a virtual machine; it is a very computationally expensive process. 
# Thus, the results are saved as a Julia data ".jld" file

In [ ]:
using Distributions
using Random
using JLD

In [ ]:
function simpler_jump(x, y, a = 1)
    theta = rand(Uniform(0, 2 * π))
    u = rand(Uniform(0, 1))

    r = a/(3 * u)^(1/3)

    xfl = r * sin(theta)
    yfl = r * cos(theta)

    (floor(Int, x + xfl + 1/2),
    floor(Int, y + yfl + 1/2))
end

function probinfected(x, y, r, p1, p2)
    if x^2 + y^2 < r^2
        return p1
    else
        return p2
    end
end

function outbreakwithjumps_varp(maxtime :: Int,
                           maxtau :: Int,
                           nextstepfcn :: Function,
                           probinfected :: Function,
                           p_arr :: Array{Float64, 1},
                           addinfectors = true) :: Tuple{Array{Int, 1},
                                                   Array{Array{Int}, 1},
                                                   Array{Array{Int}, 1},
                                                   Array{Int64, 1}}
    # Tracking the different agents
    x_start = 0
    y_start = 0

    newinfections = Array{Int64, 1}()
    infectors = [[x_start, y_start, 0, 0]]
    numinfected = [length(infectors)]
    removed = Set([[x_start, y_start]])
    r_inf = Array{Tuple{Int64, Int64}, 1}()


    for time in 1:maxtime
        ninfectors = length(infectors)
        ninfected = 0
        newinfections_now = 0

        for i in 1:ninfectors
            infector = infectors[i]
            (infector[1], infector[2]) = simpler_jump(infector[1], infector[2])
            infector[3] += 1

            inremoved = infector[1:2] in removed

            isinfecting = rand() <= probinfected(infector[1], infector[2], p_arr[1], p_arr[2], p_arr[3])

            if !inremoved && isinfecting
                push!(removed, infector[1:2])
            end

            if !inremoved && isinfecting
                newinfections_now += 1
                if !addinfectors
                    ninfected += 1
                else
                    push!(infectors, [infector[1:2]..., 0, 0])
            end

                infector[4] += 1
            end
        end

        push!(newinfections, newinfections_now)

        infectedtoremove = filter(x -> x[3] >= maxtau, infectors)
        for infector in infectedtoremove
            push!(r_inf, (infector[4], time))
        end

        filter!(x -> x[3] < maxtau, infectors)

        if addinfectors
            push!(numinfected, length(infectors))
        else
            push!(numinfected, ninfected)
        end

    end

    removed = collect(removed)

    (numinfected, infectors, removed, newinfections)

end

function getrnots(maxtime :: Int64,
                    tau :: Int64,
                    p :: Float64,
                    nextstepfcn :: Function,
                    probinfected :: Function,
                    #graphcapturetimes :: Array{Int, 1},
                    iterations = 10000,
                    addinfectors = false)
    rnots = Array{Int64, 1}()

    for i in 1:iterations
        (infections, _, _, _) = outbreakwithjumps_varp(
            maxtime, tau, nextstepfcn, probinfected, [1, p, p], addinfectors
        )

        push!(rnots, sum(infections)-1)
    end

    meanrnot = sum(rnots)/length(rnots)

    (meanrnot)
end

function getrnots_array(p_to_test, r0, τ, iterations, addinfectors)
    p_to_test = reverse(p_to_test, dims=1)
    for i in 1:length(p_to_test)
        # maxtime, tau, p, jump fcn, prob of infection function, num iterations, add infectors?
        calculatedr0 = getrnots(τ+1, τ, p_to_test[i], simpler_jump, probinfected, iterations, addinfectors)
        if round(calculatedr0, digits=2) == r0
            return p_to_test[i]
        elseif calculatedr0 < r0
            if i != 1
                return ("passed r0", p_to_test[i], p_to_test[i-1])
            else
                return ("passed r0", p_to_test[i])
            end
        end
    end

    return 0
end

function estimatep(r0 :: Int,
                    τ :: Int)

    if r0/τ > 1
        return NaN
    else
        if 2*r0/τ>1
            firstguessp = 1
        else
            firstguessp = 2 * r0/τ
        end
    end

    iterations=10_000
    addinfectors = false

    # maxtime, tau, p, jump fcn, prob of infection function, num iterations, add infectors?
    firstguessrnot = getrnots(τ+1, τ, Float64(firstguessp), simpler_jump, probinfected, iterations, addinfectors)
    if round(firstguessrnot, digits=2) == r0
        return (firstguessp, 0)
    end

    minp = 0
    maxp = firstguessp

    p = Float64
    incr = Float64
    foundrnot = false

    while !foundrnot
        incr = (maxp - minp)/9
        p_to_test = minp:incr:maxp
        returnedvalue = getrnots_array(p_to_test, r0, τ, iterations, addinfectors)
        if returnedvalue == 0
            # this means that the smallest value tested was not good enough, so we need to be even smaller
            minp = 0
            maxp = incr
        elseif length(returnedvalue) == 1
            # this means we have found a suitable p value and are done
            p = returnedvalue
            foundrnot = true
        elseif length(returnedvalue) == 2
            if maxp == 1
                # this means that the previous iteration with max p = 1 did not work out; we cannot go any higher so just return NaN.
                return NaN
            else
                # this means that the p's we tested were too small, so we will go bigger
                minp = returnedvalue[2]
                maxp = minp * 1.5
                if maxp > 1
                    # we do not want to try a p bigger than 1. that is not meaningful
                    maxp = 1
                end
            end
        elseif length(returnedvalue) == 3
            minp = returnedvalue[2]
            maxp = returnedvalue[3]
        end
    end

    return (p, incr)
end

In [ ]:
# -------------------------- EDIT INPUTS BELOW ------------------------------- #

τ_inputs = [1, 2, 3, 4, 6, 8, 10, 15, 20, 40, 60, 80, 100]
r0_inputs = 1:40

estimatedp = zeros(Float64, length(r0_inputs), length(τ_inputs))
estimatedp_err = zeros(Float64, length(r0_inputs), length(τ_inputs))

println()

t1 = time()
@Threads.threads for i in 1:length(r0_inputs) # Hyperthreading
    for j in 1:length(τ_inputs)
        idx = (i-1)*length(τ_inputs) + j
        r0 = r0_inputs[i]
        τ = τ_inputs[j]
        answer = estimatep(r0, τ)
        if length(answer) == 2
            estimatedp[i, j] = answer[1]
            estimatedp_err[i, j] = answer[2]
        else
            estimatedp[i, j] = answer
            estimatedp_err[i, j] = NaN
        end
        println(string("For R0 = ", r0, " and tau = ", τ, ", the corresponding p is ", estimatedp[i, j], "."))
        println("Completed ", idx, "/", length(r0_inputs)*length(τ_inputs), " calculations. Time elapsed is ", round(time()-t1, digits = 3), " sec.")
        println()
    end
end

t2 = time()
println(string("Total amount of time elapsed is ", t2-t1, " seconds."))

t_elapsed = t2 - t1

exportfilename = "isor0calc" # DO NOT include the file format at the end
# -------------------------------------------------------------------

fn = string(exportfilename, ".jld")
readme = "This data was generated with tau being the inner for loop (and r0 is the outer for loop). So the second data point is the first r0 value and the second tau value. Result matrix has r0 as rows and tau as column"
save(fn, "τ_inputs", τ_inputs, "r0_inputs", r0_inputs, "estimatedp", estimatedp,
    "estimatedp_err", estimatedp_err, "t_elapsed", t_elapsed)